In [ ]:
#!pip install efficientnet_pytorch

In [ ]:
!pwd

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
import albumentations as albu

import glob
from tqdm.notebook import tqdm

import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import sklearn
#from efficientnet_pytorch import EfficientNet
import pandas as pd

import cv2
from PIL import Image
from matplotlib import pyplot as plt

#modelName = 'efficientnet-b4'
#modelSize = EfficientNet.get_image_size(modelName)
torch.manual_seed(42)

In [ ]:
import pandas as pd
import glob

data = glob.glob("../input/deweight-huaweimetal/de_weight/de-weight-train/*/*.jpg")

name2class = {}
name = 6.5
for i in range(14):
    name2class[str(name)] = i
    name += 0.5
    
fileName = []
label = []
className = []
for fn in data:
    fileName.append(fn.split("/")[-1].strip(".jpg"))
    className.append(fn.split("/")[-2])
    label.append(name2class[fn.split("/")[-2]])
d = {"fileName":fileName, "Class":label,"ClassName":className}
df = pd.DataFrame(d)

df.to_csv('classes_de.csv')

In [ ]:
#!unzip -q /home/ma-user/work/data.zip

In [ ]:
df = pd.read_csv("./classes_de.csv")
#df = pd.read_csv("/home/ma-user/work/classes.csv")
fnames = df["fileName"].tolist()
classNames = df["ClassName"].tolist()
classes = df["Class"].tolist()

***Dataset***

预处理： - crop正方形之后resize (1376,1104) - > (1104,1104) - > (224,224)
        - 黑白图片normalize mean 0.5 标准差 0.5(?)

Aug:   见dataset.tfms 

训练黑白和彩色模型时需要改dataset相关部分。
现在dataset为加载彩色与黑白图片后 cat到一起 （batch_size，channel = 6，224,224）

In [ ]:
class MetalDataset(Dataset):
    def __init__(self, fileNames, labels, training = True, preproccess = False):
        #self.img_path = "/home/ma-user/work/train"
        self.training = training
        crop_size = 1104
        if training:
            self.tfms = albu.Compose([
                                        albu.CenterCrop(1104,1104),
                                        albu.Resize(224, 224), 
                                        albu.HorizontalFlip(p=0.5),
                                        albu.VerticalFlip(p=0.5),
                                        albu.Transpose(p=0.2),
                                        albu.ElasticTransform(alpha=2000,sigma=100,alpha_affine=1,p=0.2), #一般来说，alpha越小，sigma越大，产生的偏差越小，和原图越接近
                                        albu.MotionBlur(blur_limit=5,p=0.1),
                                        albu.Rotate(limit=(-180,180),p = 0.1, border_mode=cv2.BORDER_WRAP)],
                         additional_targets={
                                                'image': 'image',
                                                'gray': 'image',}
                        )      
        else:
            self.tfms = albu.Compose([
                                        albu.CenterCrop(1104,1104),
                                        albu.Resize(224, 224), 
                                                           ], 
                            additional_targets={
                                        'image': 'image',
                                        'gray': 'image',}
            )        
            
        self.fileNames = fileNames
        self.labels = labels
        self.name2class = {}
        name = 6.5
        for i in range(14):
            self.name2class[name] = torch.tensor(i)
            name += 0.5
            
    def __getitem__(self, index):
        img = cv2.imread("../input/deweight-huaweimetal/de_weight/de-weight-train/{:}/{:}.jpg".format(self.labels[index],self.fileNames[index]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2RGB)
        transformed = self.tfms(image=img, gray=gray)
        #transformed = self.tfms(image = img)
        img = transformed["image"]
        gray = transformed["gray"]
        
        img = Image.fromarray(img)   
        gray = Image.fromarray(gray) 
        img = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])(img)
        gray = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                 std=[0.5, 0.5, 0.5])])(gray)
        img = torch.cat((img,gray),0)
        return img, self.name2class[self.labels[index]]
    
    def __len__(self):
        return len(self.fileNames)

In [ ]:
MetalDataset(fnames, classNames)[0][0].shape

***Train & Eval one epoche***

In [ ]:
def trainOneEpoche(model, loader, criterion, optimizer,reg = False):
    model.train()
    runningLoss = 0
    i = 0
    y_all = []
    outputs_all = [] # For Acc
    for imgs, labels in tqdm(loader):
        imgs = imgs.to(device)
        labels= labels.to(device)
        optimizer.zero_grad()
        
        output = model(imgs).squeeze()
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        runningLoss += loss
        i+=1
        outputs_all.extend(output.cpu().detach().numpy())
        y_all.extend(labels.cpu().detach().numpy())

    acc = computeAccuracy(outputs_all, y_all, reg = reg)
    return runningLoss/i, acc

def evalOneEpoche(model, loader, criterion, reg = False):
    model.eval()
    runningLoss = 0
    i = 0
    
    y_all = []
    outputs_all = [] # For AuC
    
    with torch.no_grad():
        
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels= labels.to(device)
            output = model(imgs).squeeze()
            
            loss = criterion(output, labels)
            runningLoss += loss
            i+=1
            outputs_all.extend(output.cpu().detach().numpy())
            y_all.extend(labels.cpu().detach().numpy())

        acc = computeAccuracy(outputs_all, y_all, reg = reg)
        return runningLoss/i, acc

***Custom Loss function***

In [ ]:
class MultiFocalLoss(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)^gamma*log(pt)
    :param num_class:
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param smooth: (float,double) smooth value when cross entropy
    :param balance_index: (int) balance class index, should be specific when alpha is float
    :param size_average: (bool, optional) By default, the losses are averaged over each loss element in the batch.
    """

    def __init__(self, num_class, alpha=None, gamma=2, balance_index=-1, smooth=None, size_average=True):
        super(MultiFocalLoss, self).__init__()
        self.num_class = num_class
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = smooth
        self.size_average = size_average

        if self.alpha is None:
            self.alpha = torch.ones(self.num_class, 1)
        elif isinstance(self.alpha, (list, np.ndarray)):
            assert len(self.alpha) == self.num_class
            self.alpha = torch.FloatTensor(alpha).view(self.num_class, 1)
            self.alpha = self.alpha / self.alpha.sum()
        elif isinstance(self.alpha, float):
            alpha = torch.ones(self.num_class, 1)
            alpha = alpha * (1 - self.alpha)
            alpha[balance_index] = self.alpha
            self.alpha = alpha
        else:
            raise TypeError('Not support alpha type')

        if self.smooth is not None:
            if self.smooth < 0 or self.smooth > 1.0:
                raise ValueError('smooth value should be in [0,1]')

    def forward(self, input, target):
        logit = F.softmax(input, dim=1)

        if logit.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            logit = logit.view(logit.size(0), logit.size(1), -1)
            logit = logit.permute(0, 2, 1).contiguous()
            logit = logit.view(-1, logit.size(-1))
        target = target.view(-1, 1)

        # N = input.size(0)
        # alpha = torch.ones(N, self.num_class)
        # alpha = alpha * (1 - self.alpha)
        # alpha = alpha.scatter_(1, target.long(), self.alpha)
        epsilon = 1e-10
        alpha = self.alpha
        if alpha.device != input.device:
            alpha = alpha.to(input.device)

        idx = target.cpu().long()
        one_hot_key = torch.FloatTensor(target.size(0), self.num_class).zero_()
        one_hot_key = one_hot_key.scatter_(1, idx, 1)
        if one_hot_key.device != logit.device:
            one_hot_key = one_hot_key.to(logit.device)

        if self.smooth:
            one_hot_key = torch.clamp(
                one_hot_key, self.smooth, 1.0 - self.smooth)
        pt = (one_hot_key * logit).sum(1) + epsilon
        logpt = pt.log()

        gamma = self.gamma

        alpha = alpha[idx]
        loss = -1 * alpha * torch.pow((1 - pt), gamma) * logpt

        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss

BetterLoss 模拟regression，把各个相似类的信息互通 （单纯训练regression task结果不如分类好 ~-2.0% acc）

In [ ]:
class betterLoss(nn.Module):
    ## criterion for True label, CE loss for fuzzy label
    def __init__(self, criterion =  nn.CrossEntropyLoss(), beta = 0.1):
        super(betterLoss, self).__init__()
        self.criterion = criterion
        self.beta = beta
    def forward(self, output, labels):
        lossTrue = self.criterion(output,labels) * (1/(1+self.beta))
        labels_plus = labels.clone()
        labels_minus = labels.clone()
        for i in range(len(labels)):
            if labels[i] > 0:
                labels_minus[i]-=1
            if labels[i]< 13:
                labels_plus[i]+=1

        lossFuzzy = (self.beta/(1+self.beta))*(self.criterion(output,labels_plus) + self.criterion(output,labels_minus))
        
        return lossTrue + lossFuzzy

***Accuracy***

In [ ]:
def computeAccuracy(output,labels,reg = False):
    alpha = 0.4
    beta = 0.6
    acc_0_classes = [1]*14
    acc_0_5_classes = [1]*14
    class_num = [1]*14
    with torch.no_grad():
        score_0 = 0
        score_0_5 = 0
        output = torch.tensor(output)
        if reg:
            pred = torch.round(output) #For regression task
        else:
            _, pred = output.topk(1)
        score = 0
        for o,l in zip(pred,labels):
            l = int(l)
            o = max(0,min(13,o))
            class_num[l] += 1
            if o == l:
                score += 1
                score_0 +=1
                score_0_5 += 1
                acc_0_classes[l] += 1
            elif o == l+1 or o == l-1:
                score += beta
                score_0_5 += 1
                acc_0_5_classes[l] += 1
        acc_0_classes = [x/y for x,y in zip(acc_0_classes,class_num)]
        acc_0_5_classes = [x/y for x,y in zip(acc_0_5_classes,class_num)]
        
        return score/len(labels), score_0/len(labels), score_0_5/len(labels), acc_0_classes, acc_0_5_classes

将两个模型的结果softmax归一后 50%50%相加 我训练了黑白+彩色模型

In [ ]:
class mergeModel(nn.Module):
    def __init__(self, model1=None, model2=None):
        super(mergeModel, self).__init__()
        if model1:
            self.model1 = model1
        else:
            model = torchvision.models.__dict__['resnet152'](pretrained  = False)
            channel_in = model.fc.in_features
            model.fc = nn.Linear(channel_in, 14) #Classificiation
            self.model1 = model
        if model2:
            self.model2 = model2
        else:
            model = torchvision.models.__dict__['resnet152'](pretrained  = False)
            channel_in = model.fc.in_features
            model.fc = nn.Linear(channel_in, 14) #Classificiation
            self.model2 = model
            
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, img):
        o1 = self.model1(img[:,:3,:,:])
        o2 = self.model2(img[:,3:,:,:])
        o1 = self.softmax(o1)
        o2 = self.softmax(o2)
        out = 0.5*o1+0.5*o2
        
        return out

***Params***

In [ ]:
batch_size = 64
initial_LR = 2e-4
gamma = 0.8 #Exp scheduler gamma

epoches = 70
T_0 = 10
T_mult = 2
n_splits = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights= sklearn.utils.class_weight.compute_class_weight('balanced',np.unique(classes),np.array(classes))
#class_weights=torch.tensor(class_weights,dtype=torch.float).to(device)

#criterion = nn.CrossEntropyLoss(class_weights)
criterion = MultiFocalLoss(14, alpha=class_weights, gamma=5)
print(class_weights)
criterion = betterLoss(criterion,beta = 0.1)
#criterion = nn.MSELoss()
#eval_crit = nn.MSELoss()
eval_crit = nn.CrossEntropyLoss()
use_cv = True
preproc = True
reg = False

skf = StratifiedKFold(n_splits=n_splits, shuffle = True, random_state = 21)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelRGB = torch.load("../input/metalmodels/0.8501278772378519_trained_resnet152.pt", map_location=device)
modelGRAY = torch.load("../input/metalmodels/0.8629156010230179_trained_effnet.pt", map_location=device)
model = mergeModel(modelRGB, modelGRAY)

torch.save(model,"mergedModel.pt")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load("../input/metaltestmodels/mergedModel.pt", map_location=device)
torch.save(model.state_dict(),"mergedModel.pt")

In [ ]:
model = mergeModel()
model.load_state_dict(torch.load("mergedModel.pt"))

***Do Cross Validation***

In [ ]:
for i_train, i_val in skf.split(fnames, classes):
    max_acc = 0.84
    trainDataset = MetalDataset([fnames[i] for i in i_train],[classNames[i] for i in i_train],training = True)
    valDataset = MetalDataset([fnames[i] for i in i_val],[classNames[i] for i in i_val],training = False)
    trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batch_size, shuffle=True, num_workers=4, pin_memory=True)
    valLoader = torch.utils.data.DataLoader(valDataset, batch_size = batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    #此处为不同模型
    model = torchvision.models.__dict__['resnet152'](pretrained  = True)
    channel_in = model.fc.in_features
    model.fc = nn.Linear(channel_in, 14) #Classificiation
    #model.fc = nn.Linear(channel_in, 1) #Regression
    #modelRGB = torch.load("../input/metalmodels/0.8501278772378519_trained_resnet152.pt")
    #modelGRAY = torch.load("../input/metalmodels/0.8629156010230179_trained_effnet.pt")
    model = mergeModel(modelRGB, modelGRAY)
    #model = EfficientNet.from_pretrained(modelName)
    #channel_in = model._fc.in_features
    #model._fc = nn.Linear(channel_in, 14)
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=initial_LR)
    #scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = T_0, T_mult=T_mult, eta_min=initial_LR*0.1)
    
    TrainingLoss = []
    ValLoss = []
    ValAcc = []
    ValAcc_0 = []
    ValAcc_0_5 = []

    TrainingLoss.append([])
    ValLoss.append([])
    ValAcc.append([])
    ValAcc_0.append([])
    ValAcc_0_5.append([])

    for e in range(epoches):
            #runningtrainLoss,(train_acc,train_acc_0,train_acc_0_5,train_acc_0_classes, train_acc_0_5_classes) = trainOneEpoche(model, trainLoader, criterion, optimizer,reg=reg)
            runningvalLoss, (acc,acc_0,acc_0_5,acc_0_classes, acc_0_5_classes) = evalOneEpoche(model, valLoader, eval_crit,reg=reg)
            print("Current LR: {:}".format(scheduler.get_last_lr()))
            #print("epoche {:} TrainingLoss: {:}, Accuracy: {:}, AccuracyTrue: {:}, Accuracy+-0.5: {:}".format(e,round(float(runningtrainLoss),4), round(float(train_acc),4), round(float(train_acc_0),4), round(float(train_acc_0_5),4)))
            print("--------- EvalLoss: {:}, Accuracy: {:}, AccuracyTrue: {:}, Accuracy+-0.5: {:}".format(round(float(runningvalLoss),4),round(float(acc),4),round(float(acc_0),4),round(float(acc_0_5),4)))
            print(acc_0_classes)
            TrainingLoss[-1].append(runningtrainLoss)
            ValLoss[-1].append(runningvalLoss)
            ValAcc[-1].append(acc)
            ValAcc_0[-1].append(acc_0)
            ValAcc_0_5[-1].append(acc_0_5)
            if acc > max_acc:
                max_acc = acc
                torch.save(model, "{:}_trained_effnet.pt".format(acc))
            scheduler.step()


***Evaluate models***

In [ ]:
models = glob.glob("./*.pt")

In [ ]:
for i,(x,y,z,a,b) in enumerate(zip(TrainingLoss[0], ValLoss[0], ValAcc[0],ValAcc_0[0],ValAcc_0_5[0])):
    print("epoches: {:}, TrainingLoss:{:}, EvalLoss: {:}, Accuracy: {:}, AccuracyTrue: {:}, Accuracy+-0.5: {:}".format(i,round(float(x),6),round(float(y),4),round(float(z),4),round(float(a),4),round(float(b),4)))

In [ ]:
t = 0
for trainingLoss in TrainingLoss[-1]:
    plt.plot(list(range(epoches)), trainingLoss)
    t+= trainingLoss[-1]
print(t/len(TrainingLoss))
plt.show()

In [ ]:
t = 0
for valLoss in ValLoss:
    plt.plot(list(range(epoches)), valLoss)
    t+= valLoss[-1]
print(t/len(TrainingLoss))
plt.show()

In [ ]:
#t = 0
#for aucs in ValAuc:
#    plt.plot(list(range(epoches)), aucs)
#    t+= aucs[-1]
#print(t/len(ValAuc))
#plt.show()

In [ ]:
t = 0
for accs in ValAcc:
    plt.plot(list(range(epoches)), accs)
    t+= accs[-1]
print(t/len(ValAcc))
plt.show()